宏的调用
```julia
@macroname expr1 expr2 ... # n 个参数表达式
@macroname(expr1, expr2, ...) # n 个参数表达式
@macroname (expr1, expr2, ...) # 1 个参数表达式(expr1, expr2, ...)
```
在数组字面量（或推导式）上调用宏的另一种方法是不使用括号直接并列两者。在这种情况下，数组将是唯一的传给宏的表达式。以下语法等价（且与 `@name [a b] * v` 不同）：
```julia
@macroname[a b] * v
@macroname([a b]) * v
```

**宏把它们的参数作为表达式、字面量或符号接收**。浏览宏参数的一种方法是在宏的内部调用 show 函数：

In [1]:
macro showarg(x)
    show(x)
    :(x+1)
    # ... remainder of macro, returning an expression
    # ... 宏的剩余部分，返回一个表达式    
end

@showarg (macro with 1 method)

In [2]:
x=1
@showarg(a)

:a

2

In [4]:
x=3
@showarg 1+1

:(1 + 1)

4

In [7]:
@showarg(println("Yo!"))

:(println("Yo!"))

4

In [8]:
@showarg if x>0 x=x+1 else x=x-1 end

:(if x > 0
      #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:1 =#
      x = x + 1
  else
      #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:1 =#
      x = x - 1
  end)

4

In [9]:
@showarg(quote if x>0 x=x+1 else x=x-1 end end)

:($(Expr(:quote, quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:1 =#
    if x > 0
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:1 =#
        x = x + 1
    else
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:1 =#
        x = x - 1
    end
end)))

4

In [ ]:
@showarg(x=1; if x>0 x=x+1 else x=x-1 end)
"""Error: The function `@showarg` exists, 
but no method is defined for this combination of argument types."""
nothing

In [11]:
@showarg(quote x=1; if x>0 x=x+1 else x=x-1 end end)

:($(Expr(:quote, quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:1 =#
    x = 1
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:1 =#
    if x > 0
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:1 =#
        x = x + 1
    else
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:1 =#
        x = x - 1
    end
end)))

4

In [12]:
@showarg(:(x=1; if x>0 x=x+1 else x=x-1 end))

:($(Expr(:quote, quote
    x = 1
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:1 =#
    if x > 0
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:1 =#
        x = x + 1
    else
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:1 =#
        x = x - 1
    end
end)))

4

除了给定的参数列表，每个宏都会传递名为 `__source__` 和 `__module__` 的额外参数。

+ 参数 `__source__` 提供 @ 符号在宏调用处的解析器位置的相关信息（以 LineNumberNode 对象的形式）。这使得宏能包含更好的错误诊断信息，其通常用于日志记录、字符串解析器宏和文档，比如，用于实现 `@__LINE__`、`@__FILE__` 和 `@__DIR__` 宏。引用 `__source__.line` 和 `__source__.file` 即可访问位置信息.<br><br>
+ 参数 `__module__` 提供宏调用展开处的上下文相关信息（以 `Module` 对象的形式）。这允许宏查找上下文相关的信息，比如现有的绑定，或者将值作为附加参数插入到一个在当前模块中进行自我反射的运行时函数调用中。

In [13]:
macro __LOCATION__() 
    return QuoteNode(__source__)
end
dump(@__LOCATION__())

LineNumberNode
  line: Int64 4
  file: Symbol e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X13sZmlsZQ==.jl


### https://docs.juliacn.com/latest/manual/metaprogramming/#宏与派发

与 Julia 函数一样，宏也是泛型的。由于多重派发，这意味着宏也能有多个方法定义：

In [1]:
macro m end

macro m(args...)
    println("$(length(args)) arguments")
end

macro m(x,y)
    println("Two arguments")
end

@m (macro with 2 methods)

In [2]:
@m "asd"

1 arguments


In [3]:
@m 1 2

Two arguments


但是应该记住，**宏派发基于传递给宏的 AST 的类型，而不是 AST 在运行时进行求值的类型：**

In [4]:
macro m(::Int)
    println("An Integer")
end

@m (macro with 3 methods)

In [5]:
@m 2

An Integer


In [6]:
x = 2

2

In [7]:
@m x

1 arguments
